# Example: QAT Resnet18 Optimization

This is the notebook described in the [TREx blog post](https://developer.nvidia.com/blog/exploring-tensorrt-engines-with-trex).

## Load JSON Files

In [ ]:
import os
import IPython
from ipywidgets import widgets
from trex import *

# Configure a wider output (for the wide graphs)
set_wide_display()

def create_plan(engine_path: str, engine_name):
    plan = EnginePlan(
        f'{engine_path}.graph.json',
        f'{engine_path}.profile.json',
        f"{engine_path}.profile.metadata.json",
        name = engine_name)
    return plan


ipynb_path = os.path.dirname(os.path.realpath("__file__"))

examples_dir = os.path.join(ipynb_path, "A100")
engines_info = {
    "ResNet18-FP32": "fp32/resnet.onnx.engine",
    "ResNet18-FP16": "fp16/resnet.onnx.engine",
    "ResNet18-QAT":  "qat/resnet-qat.onnx.engine",
    "ResNet18-QAT+Residual": "qat-residual/resnet-qat-residual.onnx.engine",
    "ResNet18-QAT+Residual+GAP": "qat-residual-qgap/resnet-qat-residual-qgap.onnx.engine",
}


plans = [create_plan(os.path.join(examples_dir, engine_path), engine_name) for  engine_name, engine_path in engines_info.items()]
compare_engines_summaries_tbl(plans, orientation='vertical')

In [ ]:
compare_engines_overview(plans)

In [ ]:
compare_engines_layer_latencies(
    plans[1], plans[3],
    # Allow for 3% error grace threshold when color highlighting performance differences
    threshold=0.03,
    # Inexact matching uses only the layer's first input and output to match to other layers.
    exact_matching=True)

In [ ]:
for plan in plans:
    graph = to_dot(plan, layer_type_formatter, display_regions=True, expand_layer_details=True)
    render_dot(graph, plan.name, 'svg')